<a href="https://colab.research.google.com/github/notKguin/MMK-Informservice/blob/main/final_project/project/MMK_Informservice_project_DP_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import gradio as gr
import pandas as pd
import numpy as np
import pickle

# Загрузка датасетов
chugun_df = pd.read_csv("chugun.csv")
coke_df = pd.read_csv("coke.csv")

# Загрузка моделей
with open("gb_model_chugun.pkl", "rb") as f:
    gb_model_chugun = pickle.load(f)
with open("gb_model_coke.pkl", "rb") as f:
    gb_model_coke = pickle.load(f)

In [21]:
import gradio as gr
import pandas as pd
import numpy as np
import pickle

# Загрузка датасетов
chugun_df = pd.read_csv("chugun.csv")
coke_df = pd.read_csv("coke.csv")

# Загрузка моделей
with open("gb_model_chugun.pkl", "rb") as f:
    gb_model_chugun = pickle.load(f)
with open("gb_model_coke.pkl", "rb") as f:
    gb_model_coke = pickle.load(f)

# Функция для получения случайной строки из датасета
def get_random_sample(dataset):
    return dataset.sample(1)

# Функция для форматирования признаков с учетом ожидаемых признаков модели
def format_features(sample, target_column, model):
    # Удаляем целевую переменную из признаков
    features = sample.drop(columns=[target_column])
    feature_names = list(features.columns)

    # Проверяем, совпадают ли признаки с ожидаемыми моделью
    expected_features = model.feature_names_in_ if hasattr(model, 'feature_names_in_') else feature_names
    if set(feature_names) != set(expected_features):
        missing_features = set(expected_features) - set(feature_names)
        extra_features = set(feature_names) - set(expected_features)
        if missing_features:
            raise ValueError(f"Датасет не содержит ожидаемые признаки: {missing_features}")
        if extra_features:
            # Удаляем лишние признаки
            features = features[expected_features]
            feature_names = expected_features

    feature_values = features.values[0]
    return feature_names, feature_values

# Основная функция для предсказания
def predict(model_choice):
    if model_choice == "Производство чугуна":
        dataset = chugun_df
        target_column = "Количество чугуна (т)"
        model = gb_model_chugun
    else:
        dataset = coke_df
        target_column = "Удельный расход кокса (кг/т)"
        model = gb_model_coke

    # Получаем случайную строку
    sample = get_random_sample(dataset)
    feature_names, feature_values = format_features(sample, target_column, model)
    actual_value = sample[target_column].values[0]

    # Преобразуем входные данные в массив
    input_data = np.array([feature_values])

    # Предсказание
    prediction = model.predict(input_data)[0]

    # Расчет процентной ошибки
    error_percent = abs((prediction - actual_value) / actual_value) * 100

    # Формирование результата
    result = f"Предсказанное значение: {prediction:.2f}\n"
    result += f"Реальное значение: {actual_value:.2f}\n"
    result += f"Процент ошибки: {error_percent:.2f}%"

    return (
        gr.Dataframe(headers=["Признак"], value=[[name] for name in feature_names]),
        gr.Dataframe(headers=["Значение"], value=[[value] for value in feature_values]),
        result
    )

# Создание интерфейса Gradio
with gr.Blocks() as demo:

    # Выбор модели
    model_choice = gr.Dropdown(
        choices=["Производство чугуна", "Удельный расход кокса"],
        label="Выберите модель",
        value="Производство чугуна"
    )

    # Вывод признаков
    with gr.Row():
      feature_display = gr.Dataframe(headers=["Признак"], value=[])
      value_display = gr.Dataframe(headers=["Значение"], value=[])

    # Кнопка предсказания
    predict_button = gr.Button("Предсказать")

    # Вывод результата
    output = gr.Textbox(label="Результат предсказания")

    # Логика предсказания
    predict_button.click(
        fn=predict,
        inputs=model_choice,
        outputs=[feature_display, value_display, output]
    )

# Запуск приложения
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4c4bc90749ae538114.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
